## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-24-06-50-43 +0000,nypost,"Russian plane with nearly 50 aboard, including...",https://nypost.com/2025/07/24/world-news/russi...
1,2025-07-24-06-46-24 +0000,nyt,A Timeline of the Latest Conflict Between Camb...,https://www.nytimes.com/2025/07/24/world/asia/...
2,2025-07-24-06-32-40 +0000,nyt,Chinese Hackers Are Exploiting Flaws in Widely...,https://www.nytimes.com/2025/07/23/world/asia/...
3,2025-07-24-06-24-00 +0000,wsj,German Consumer Confidence Hit by Economic Unc...,https://www.wsj.com/economy/consumers/german-c...
4,2025-07-24-06-23-46 +0000,bbc,Tories must 'offer hope' in face of Reform thr...,https://www.bbc.com/news/articles/cy4yj3rpjm8o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,61
64,new,19
190,deal,18
121,trade,16
673,he,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
55,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...,147
136,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...,130
324,2025-07-23-09-24-00 +0000,wsj,President Trump announced a trade deal with Ja...,https://www.wsj.com/economy/trade/u-s-japan-st...,128
218,2025-07-23-18-03-36 +0000,nyt,How Japan Clinched a Trade Deal With Trump Jus...,https://www.nytimes.com/2025/07/23/business/tr...,122
91,2025-07-23-23-26-00 +0000,wsj,The Trump administration issued a new AI actio...,https://www.wsj.com/tech/ai/trump-pledges-move...,116


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
55,147,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...
136,69,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...
100,61,2025-07-23-23-00-49 +0000,nypost,New trove of disturbing Bryan Kohberger docume...,https://nypost.com/2025/07/23/us-news/new-trov...
228,49,2025-07-23-16-59-32 +0000,nypost,"Emanuel, Brigitte Macron sue Candace Owens for...",https://nypost.com/2025/07/23/media/candace-ow...
95,48,2025-07-23-23-17-00 +0000,wsj,Tulsi Gabbard and the White House alleged form...,https://www.wsj.com/politics/policy/tulsi-gabb...
24,44,2025-07-24-04-13-46 +0000,nypost,Ozzy Osbourne memorabilia — including custom o...,https://nypost.com/2025/07/24/entertainment/oz...
33,41,2025-07-24-03-03-24 +0000,nypost,Appeals court blocks Trump’s effort to end bir...,https://nypost.com/2025/07/23/us-news/appeals-...
204,41,2025-07-23-19-08-00 +0000,wsj,Justice Department told Trump in May that his ...,https://www.wsj.com/politics/justice-departmen...
81,36,2025-07-23-23-44-11 +0000,nypost,Kamala Harris’ ex-lover Willie Brown warns her...,https://nypost.com/2025/07/23/us-news/kamala-h...
184,35,2025-07-23-19-45-05 +0000,latimes,Tech companies want to move fast. Trump's 'AI ...,https://www.latimes.com/business/story/2025-07...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
